In [1]:
DATA_NAME = 'syntheticMRI2D-coronal' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_vertical_wavelet_coronal_syntheticMRI2D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,3,4,5,6,7,8,9
3,6627953.83,132.46,-1.39,-22.29,1.82,-1.06,0.01
4,132.46,2351329.63,47.15,6.09,9.32,-1.37,-0.04
5,-1.39,47.15,577948.63,2.10,-1.95,0.99,0.03
6,-22.29,6.09,2.10,141859.82,-1.16,-0.22,0.10
7,1.82,9.32,-1.95,-1.16,20325.35,0.09,0.00
8,-1.06,-1.37,0.99,-0.22,0.09,1470.36,0.00
9,0.01,-0.04,0.03,0.10,0.00,0.00,65.57


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

,3,4,5,6,7,8,9
3,1.00000,0.00003,-0.00000,-0.00002,0.00000,-0.00001,0.00000
4,0.00003,1.00000,0.00004,0.00001,0.00004,-0.00002,-0.00000
5,-0.00000,0.00004,1.00000,0.00001,-0.00002,0.00003,0.00000
6,-0.00002,0.00001,0.00001,1.00000,-0.00002,-0.00002,0.00003
7,0.00000,0.00004,-0.00002,-0.00002,1.00000,0.00002,0.00000
8,-0.00001,-0.00002,0.00003,-0.00002,0.00002,1.00000,0.00000
9,0.00000,-0.00000,0.00000,0.00003,0.00000,0.00000,1.00000


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

202.03206623358335

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[6.30194891e+12 7.66997626e+11 4.45943714e+10 2.51091774e+09
 4.58495717e+07 1.80280863e+05 3.00510315e-28]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.997643,-0.067040,-0.014203,-0.003460,-0.000495,-0.000036,-0.000002
1,0.066152,0.996317,-0.053149,-0.012156,-0.001730,-0.000126,-0.000006
2,0.017391,0.051179,0.996337,-0.065679,-0.008720,-0.000628,-0.000028
3,0.005367,0.015151,0.064388,0.996570,-0.049308,-0.003471,-0.000154
4,0.001022,0.002878,0.011740,0.048447,0.998083,-0.036507,-0.001615
5,0.000111,0.000312,0.001267,0.005173,0.036177,0.998334,-0.044640
6,0.000010,0.000028,0.000113,0.000461,0.003223,0.044550,0.999002


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0023569499424143103,
 0.0036825853950568366,
 0.0036626122977779785,
 0.0034296528107170543,
 0.0019166860723501822,
 0.0016663746614500319,
 0.0009981674141141816]